In [1]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [2]:
dfOscar = pd.read_csv("the_oscar_award.csv")
# an exmaple year
year = 2018
dfOscar1Yr = dfOscar[dfOscar['year_film']==year]
# dfOscar1Yr[['film','winner']].groupby('film').any().shape
dfOscar1Yr

,year_film,year_ceremony,ceremony,category,name,film,winner
10142,2018,2019,91,ACTOR IN A LEADING ROLE,Christian Bale,Vice,False
10143,2018,2019,91,ACTOR IN A LEADING ROLE,Bradley Cooper,A Star Is Born,False
10144,2018,2019,91,ACTOR IN A LEADING ROLE,Willem Dafoe,At Eternity's Gate,False
10145,2018,2019,91,ACTOR IN A LEADING ROLE,Rami Malek,Bohemian Rhapsody,True
10146,2018,2019,91,ACTOR IN A LEADING ROLE,Viggo Mortensen,Green Book,False
...,...,...,...,...,...,...,...
10262,2018,2019,91,WRITING (Original Screenplay),Written by Adam McKay,Vice,False
10263,2018,2019,91,HONORARY AWARD,Marvin Levy,NaN,True
10264,2018,2019,91,HONORARY AWARD,Lalo Schifrin,NaN,True
10265,2018,2019,91,HONORARY AWARD,Cicely Tyson,NaN,True


In [290]:
dfMovies = pd.read_csv("movie_lense/movies.csv")
reWithYr = r'.*\ *\([0-9]+.*\)'
# re1Yr = r'.*\ \(%d\)'%(year) 
dfMovies['film'] = dfMovies['title'].str.replace(r"\ \([0-9]+\)", "")

dfMovies['year_film'] = dfMovies[dfMovies['title'].str.match(reWithYr) == True]['title'].str.replace(r'.*\ *\(',"")
dfMovies['year_film'] = dfMovies['year_film'].str.slice(start=0, stop=4, step=1)
dfMovies.at[35140, 'year_film'] = '1957' # 1 special case
dfMovies['year_film'] = dfMovies['year_film'].astype(float)

In [291]:
dfOscarMovies = dfOscar.merge(dfMovies, left_on='film', right_on='film')
# year_film difference (between differernt dataset) should not exceed 1 year
dfOscarMovies = dfOscarMovies[abs(dfOscarMovies['year_film_x'] - dfOscarMovies['year_film_y']) <= 1]
dfOscarMovies

,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y
2,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,25768,Sadie Thompson (1928),Drama,1928.0
7,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,True,87290,Two Arabian Knights (1927),Adventure|Comedy|Romance,1927.0
8,1927,1928,1,DIRECTING (Comedy Picture),Ted Wilde,Speedy,False,32551,Speedy (1928),Action|Comedy,1928.0
9,1927,1928,1,ENGINEERING EFFECTS,Roy Pomeroy,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
10,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0


In [292]:
ambiCheck = dfOscarMovies[['year_film_y', 'title']].groupby('title').apply(lambda x: len(list(set(x['year_film_y'])))) # no ambiguity
ambiCheck2 = dfOscarMovies[['year_film_y', 'title']].groupby('title').apply(lambda x: list(set(x['year_film_y'])))
ambiCheck2[ambiCheck[ambiCheck != 1].index]
# dfOscarMovies[dfOscarMovies['film']=='Alice in Wonderland']  # same-name, different-year movies

Series([], dtype: object)

In [293]:
dfRatings = pd.read_csv('movie_lense/ratings.csv')
import datetime
import time
# d = datetime.datetime(1970, 1, 1, 0)
# print(time.mktime(d.timetuple()))
# dfRatings[dfRatings['timestamp']< 18000] # no rating before 1970-1-1 (Unix Epoch)

In [319]:
def get_preRatings(df, rating_df, mode='mode'):
    # collect the ratings before Feb in this year
    d = datetime.datetime(df['year_ceremony'], 2, 1, 0)
    timestamp1 = time.mktime(d.timetuple()) # local time
    if mode == 'mode':
        output = rating_df[(rating_df['timestamp']<timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mode().mean()
    else:
        output = rating_df[(rating_df['timestamp']<timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mean()
    print(df['movieId'])
    return output # return mode/mean
preRatingsMode = lambda x: get_preRatings(x, dfRatings)
preRatingsMean = lambda x: get_preRatings(x, dfRatings, 'mean')
dfOM_1970 = dfOscarMovies[dfOscarMovies['year_film_x']>=1970] # for get the preRatings, use the dataset after 1970
dfOM_1970['preRatings_mode'] = dfOM_1970.apply(preRatingsMode, axis=1) 
dfOM_1970['preRatings_mean'] = dfOM_1970.apply(preRatingsMean, axis=1) 
dfOM_1970.to_csv('oscar_movies_preRat.csv')
dfOM_1970

838
26958
838
26958
261
261
261
5777
5777
5777
192385
192385
192385
192385
192385
192385
192385
192385
45880
4132
132532
2640
2640
2640
2640
26491
2779
2779
2779
2779
2779
2779
2779
2779
2779
1199
1199
6331
1224
1224
1224
3723
3723
1411
1411
1411
1411
5838
2429
1277
1277
1277
1277
1277
130073
97057
74789
74789
74789
4308
4308
4308
4308
4308
4308
4308
4308
1345
1345
8025
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
7
7
2476
41
41
1688
1688
170611
2112
187273
137857
82459
82459
82459
82459
82459
82459
82459
82459
82459
82459
74266
74266
74266
3201
3201
3201
3201
4296
4296
4296
4296
4296
4296
4296
1272
1272
1272
1272
1272
1272
1272
1272
1272
1272
8757
8757
8757
3037
7111
7111
7111
7111
6247
6247
6247
6247
78742
2520
2520
2520
2520
2520
2520
2520
2520
2520
2520
6849
6849
6849
6849
3066
3066
3066
3066
3066
8272
8272
74547
74547
74547
4282
7836
7836
7836
141454
52528
118

1704
1704
1704
1633
1747
1747
1784
1784
1784
1784
1784
1784
1784
1729
1693
1693
1693
1693
1673
1673
1673
1733
1617
1617
1617
1617
1617
1617
1617
1617
1617
1614
1653
1730
1730
1730
1730
1780
4217
6667
4459
1608
1608
1569
1566
1552
1552
95654
1573
1676
1466
1701
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2333
2333
2333
2439
2439
2329
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2336
2336
2336
2336
2336
2336
2336
2272
2442
2442
1810
1810
2390
2321
2321
2321
2297
2297
2314
2337
2538
101062
3609
1912
1912
1907
2431
1917
1917
1917
1917
1881
2384
190219
4454
1883
3129
3129
2858
2858
2858
2858
2858
2858
2858
2858
3160
3160
3160
3118
2996
2996
2908
2908
3186
2997
2997
2997
3155
3155
3081
3081
3081
3163
3163
3163
3163
3185
3181
2677
7786
2824
3989
2683
3156
2587
3179
3114
2687
186343
2959
3157
2599
4021
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
4022
4022
4017
4017
3993
3993
3993
4036
4036
4034
4034
4034
4034
4034
3408
3408
3408
3408
3408
4014
401

166705
166705
166705
161582
161582
161582
161582
165551
165551
165551
165551
165551
165551
161966
163959
178861
166486
166486
166486
155659
155659
166643
166643
166643
162578
162578
166461
166461
160848
152081
164179
164179
164179
164179
164179
164179
164179
164179
143367
164981
135143
135143
164019
167064
160874
160289
164917
169784
169788
169810
167870
156944
151113
151113
164949
165709
158972
135569
135536
166635
166635
165141
152782
143859
168750
165413
168752
168748
160718
169532
167036
169530
169534
162598
162598
160980
166528
166528
122922
133771
166788
168492
168492
168492
168492
181315
181315
181315
181315
181315
181315
168250
168250
168250
168250
179817
179817
179817
179817
179817
179817
179815
177651
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
182825
178061
178061
178061
177615
177615
177615
177615
177615
180497
180497
168608
168608
168608
168608
168418
181671
161644
177765
161644
177765
180987
179133
176371
176371
176371
176371

C:\Users\19178\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


26958
838
26958
261
261
261
5777
5777
5777
192385
192385
192385
192385
192385
192385
192385
192385
45880
4132
132532
2640
2640
2640
2640
26491
2779
2779
2779
2779
2779
2779
2779
2779
2779
1199
1199
6331
1224
1224
1224
3723
3723
1411
1411
1411
1411
5838
2429
1277
1277
1277
1277
1277
130073
97057
74789
74789
74789
4308
4308
4308
4308
4308
4308
4308
4308
1345
1345
8025
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
1721
118916
7
7
2476
41
41
1688
1688
170611
2112
187273
137857
82459
82459
82459
82459
82459
82459
82459
82459
82459
82459
74266
74266
74266
3201
3201
3201
3201
4296
4296
4296
4296
4296
4296
4296
1272
1272
1272
1272
1272
1272
1272
1272
1272
1272
8757
8757
8757
3037
7111
7111
7111
7111
6247
6247
6247
6247
78742
2520
2520
2520
2520
2520
2520
2520
2520
2520
2520
6849
6849
6849
6849
3066
3066
3066
3066
3066
8272
8272
74547
74547
74547
4282
7836
7836
7836
141454
52528
118778


1704
1704
1704
1633
1747
1747
1784
1784
1784
1784
1784
1784
1784
1729
1693
1693
1693
1693
1673
1673
1673
1733
1617
1617
1617
1617
1617
1617
1617
1617
1617
1614
1653
1730
1730
1730
1730
1780
4217
6667
4459
1608
1608
1569
1566
1552
1552
95654
1573
1676
1466
1701
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2028
2333
2333
2333
2439
2439
2329
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2396
2336
2336
2336
2336
2336
2336
2336
2272
2442
2442
1810
1810
2390
2321
2321
2321
2297
2297
2314
2337
2538
101062
3609
1912
1912
1907
2431
1917
1917
1917
1917
1881
2384
190219
4454
1883
3129
3129
2858
2858
2858
2858
2858
2858
2858
2858
3160
3160
3160
3118
2996
2996
2908
2908
3186
2997
2997
2997
3155
3155
3081
3081
3081
3163
3163
3163
3163
3185
3181
2677
7786
2824
3989
2683
3156
2587
3179
3114
2687
186343
2959
3157
2599
4021
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
3578
4022
4022
4017
4017
3993
3993
3993
4036
4036
4034
4034
4034
4034
4034
3408
3408
3408
3408
3408
4014
401

166705
166705
161582
161582
161582
161582
165551
165551
165551
165551
165551
165551
161966
163959
178861
166486
166486
166486
155659
155659
166643
166643
166643
162578
162578
166461
166461
160848
152081
164179
164179
164179
164179
164179
164179
164179
164179
143367
164981
135143
135143
164019
167064
160874
160289
164917
169784
169788
169810
167870
156944
151113
151113
164949
165709
158972
135569
135536
166635
166635
165141
152782
143859
168750
165413
168752
168748
160718
169532
167036
169530
169534
162598
162598
160980
166528
166528
122922
133771
166788
168492
168492
168492
168492
181315
181315
181315
181315
181315
181315
168250
168250
168250
168250
179817
179817
179817
179817
179817
179817
179815
177651
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
180031
182825
178061
178061
178061
177615
177615
177615
177615
177615
180497
180497
168608
168608
168608
168608
168418
181671
161644
177765
161644
177765
180987
179133
176371
176371
176371
176371
176371

C:\Users\19178\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y,preRatings_mode,preRatings_mean
160,1996,1997,69,COSTUME DESIGN,Ruth Myers,Emma,False,838,Emma (1996),Comedy|Drama|Romance,1996.0,5.0,4.076200
161,1996,1997,69,COSTUME DESIGN,Ruth Myers,Emma,False,26958,Emma (1996),Romance,1996.0,NaN,NaN
166,1996,1997,69,MUSIC (Original Musical or Comedy Score),Rachel Portman,Emma,True,838,Emma (1996),Comedy|Drama|Romance,1996.0,5.0,4.076200
167,1996,1997,69,MUSIC (Original Musical or Comedy Score),Rachel Portman,Emma,True,26958,Emma (1996),Romance,1996.0,NaN,NaN
225,1994,1995,67,ACTRESS IN A LEADING ROLE,Winona Ryder,Little Women,False,261,Little Women (1994),Drama,1994.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0,3.5,4.000000
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0,4.0,3.758564
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0,4.0,3.483051
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,3.5,3.464356


In [320]:
dfOscarMovies

,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y
2,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,25768,Sadie Thompson (1928),Drama,1928.0
7,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,True,87290,Two Arabian Knights (1927),Adventure|Comedy|Romance,1927.0
8,1927,1928,1,DIRECTING (Comedy Picture),Ted Wilde,Speedy,False,32551,Speedy (1928),Action|Comedy,1928.0
9,1927,1928,1,ENGINEERING EFFECTS,Roy Pomeroy,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
10,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0
...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0


In [329]:
import statsmodels.api as sm
dfOM_1970 = pd.read_csv('oscar_movies_preRat.csv')
dfOM_1970 = dfOM_1970.dropna()
numOfAwards = dfOM_1970[['movieId', 'winner']].groupby('movieId').count()
# mode of preRatings
preRat = dfOM_1970[['movieId', 'preRatings_mode']].groupby('movieId').mean()
model = sm.OLS(numOfAwards, preRat).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 winner   R-squared (uncentered):                   0.550
Model:                            OLS   Adj. R-squared (uncentered):              0.549
Method:                 Least Squares   F-statistic:                              743.0
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                   1.48e-107
Time:                        00:59:27   Log-Likelihood:                         -1406.2
No. Observations:                 610   AIC:                                      2814.
Df Residuals:                     609   BIC:                                      2819.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [330]:
# mean of preRatings
preRat = dfOM_1970[['movieId', 'preRatings_mean']].groupby('movieId').mean()
model = sm.OLS(numOfAwards, preRat).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 winner   R-squared (uncentered):                   0.551
Model:                            OLS   Adj. R-squared (uncentered):              0.551
Method:                 Least Squares   F-statistic:                              748.4
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                   4.39e-108
Time:                        00:59:29   Log-Likelihood:                         -1405.0
No. Observations:                 610   AIC:                                      2812.
Df Residuals:                     609   BIC:                                      2816.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [344]:
# add other control variables
X = dfOM_1970[['movieId', 'preRatings_mean', 'year_ceremony', 'category']].groupby('movieId').apply(lambda x: x.iloc[0]) # Index doesn't matter.
X = pd.get_dummies(X, columns=['category'])
model = sm.OLS(numOfAwards, X.iloc[:,1:]).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 winner   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.377
Method:                 Least Squares   F-statistic:                     12.50
Date:                Fri, 18 Sep 2020   Prob (F-statistic):           1.58e-47
Time:                        01:18:18   Log-Likelihood:                -1265.9
No. Observations:                 610   AIC:                             2598.
Df Residuals:                     577   BIC:                             2743.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------

In [347]:
def get_postRatings(df, rating_df, mode='mode'):
    # collect the ratings after March in this year
    d = datetime.datetime(max(1970,df['year_ceremony']), 3, 31, 0)
    print(d)
    timestamp1 = time.mktime(d.timetuple()) # local time
    if mode == 'mode':
        output = rating_df[(rating_df['timestamp']>timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mode().mean()
    else:
        output = rating_df[(rating_df['timestamp']>timestamp1) & (rating_df['movieId']==df['movieId'])]['rating'].mean()
    print(df['movieId'])
    return output # return mode/mean
postRatingsMode = lambda x: get_postRatings(x, dfRatings)
postRatingsMean = lambda x: get_postRatings(x, dfRatings, 'mean')
dfOscarMovies['postRatings_mode'] = dfOscarMovies.apply(postRatingsMode, axis=1) 
dfOscarMovies['postRatings_mean'] = dfOscarMovies.apply(postRatingsMean, axis=1) 
dfOscarMovies.to_csv('oscar_movies_postRat.csv')
dfOscarMovies

1970-03-31 00:00:00
25768
1970-03-31 00:00:00
87290
1970-03-31 00:00:00
32551
1970-03-31 00:00:00
1925
1970-03-31 00:00:00
1925
1970-03-31 00:00:00
79874
1970-03-31 00:00:00
142644
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
87954
1970-03-31 00:00:00
85576
1970-03-31 00:00:00
85576
1970-03-31 00:00:00
122585
1970-03-31 00:00:00
122585
1970-03-31 00:00:00
124795
1970-03-31 00:00:00
124795
1970-03-31 00:00:00
124795
1970-03-31 00:00:00
73967
1970-03-31 00:00:00
95563
1970-03-31 00:00:00
83833
1970-03-31 00:00:00
78224
1970-03-31 00:00:00
78224
1970-03-31 00:00:00
72897
1970-03-31 00:00:00
123000
1970-03-31 00:00:00
83376
1970-03-31 00:00:00
83376
1970-03-31 00:00:00
83376
1970-03-31 00:00:00
88818
1970-03-31 00:00:00
88818
1970-03-31 00:00:00
8518
1970-03-31 00:00:00
8518
1970-03-31 00:00:00
124595
1970-03-31 00:00:00
87595
1970-03-31 00:00:00
122001
1970-03-31 00:00:00
1927
1970-03-31 00:00:00
1927
1970-03-31 00:00:00
1927
1970-03-31

944
1970-03-31 00:00:00
25842
1970-03-31 00:00:00
25842
1970-03-31 00:00:00
25833
1970-03-31 00:00:00
32349
1970-03-31 00:00:00
32349
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
36553
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25841
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
25835
1970-03-31 00:00:00
119617
1970-03-31 00:00:00
134732
1970-03-31 00:00:00
134732
1970-03-31 00:00:00
134732
1970-03-31 00:00:00
60343
1970-03-31 00:00:00
181193
1970-03-31 00:00:00
86620
1970-03-31 00:00:00
112307
1970-03-31 00:00:00
163350
1970-03-31 00:00:00
163350
1970-03-31 00:00:00
151137
1970-03-31 00:00:00
81857
1970-03-31 00:00:00
81857
1970-03-31 00:00:00
81583
1970-03-31 00:00:00
594
1970-03-31 00:00:00
975
1970-03-31 00:00:00
47723
1970-03-31 00:00:00
1066
1970-03-31 00:00:00
162716
197

25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
25870
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
923
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
1935
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
79035
1970-03-31 00:00:00
2206
1970-03-31 00:00:00
2206
1970-03-31 00:00:00
2206
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
33001
1970-03-31 00:00:00
25868
1970-03-31 00:00:00
25868
1970-03-31 

9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
9014
1970-03-31 00:00:00
25907
1970-03-31 00:00:00
25907
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
3435
1970-03-31 00:00:00
8461
1970-03-31 00:00:00
8461
1970-03-31 00:00:00
123246
1970-03-31 00:00:00
123246
1970-03-31 00:00:00
85450
1970-03-31 00:00:00
85450
1970-03-31 00:00:00
85450
1970-03-31 00:00:00
86927
1970-03-31 00:00:00
124226
1970-03-31 00:00:00
124226
1970-03-31 00:00:00
124226
1970-03-31 00:00:00
147424
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
25899
1970-03-31 00:00:00
160804
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
83825
1970-03-31 00:00:00
171197
1970-03-31 00:00:00
171197
1970-03-31 0

62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
62514
1970-03-31 00:00:00
69773
1992-03-31 00:00:00
5838
1970-03-31 00:00:00
79925
1970-03-31 00:00:00
79925
1970-03-31 00:00:00
86337
1970-03-31 00:00:00
86337
1970-03-31 00:00:00
86337
1970-03-31 00:00:00
5440
1970-03-31 00:00:00
41762
1970-03-31 00:00:00
3606
1970-03-31 00:00:00
81859
1970-03-31 00:00:00
80290
1970-03-31 00:00:00
1026
1970-03-31 00:00:00
143211
1970-03-31 00:00:00
2430
1999-03-31 00:00:00
2429
1970-03-31 00:00:00
70204
1970-03-31 00:00:00
30808
1970-03-31 00:00:00
8491
1970-03-31 00:00:00
42546
1970-03-31 00:00:00
44100
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1991-03-31 00:00:00
1277
1970-03-31 00:00:00
3088
1970-03-31 00:00:00
3088
1970-03-31 00:00:00
934
1970-03-31 00:00:00
934
1970-03-31 00:00:00
934
1970-03-31 00:00:00
54121
1970-03-31 00:00:00
54121
1970-03-31 00:00:00
54121
1970-03-31 00:00:00
83343

...

41997
2006-03-31 00:00:00
41997
2006-03-31 00:00:00
41997
2006-03-31 00:00:00
41997
2006-03-31 00:00:00
38198
2006-03-31 00:00:00
33154
2006-03-31 00:00:00
34153
2006-03-31 00:00:00
49225
2006-03-31 00:00:00
157883
2006-03-31 00:00:00
158013
2006-03-31 00:00:00
186981
2006-03-31 00:00:00
41527
2006-03-31 00:00:00
44204
2006-03-31 00:00:00
115881
2006-03-31 00:00:00
95377
2006-03-31 00:00:00
53318
2006-03-31 00:00:00
55207
2006-03-31 00:00:00
126096
2006-03-31 00:00:00
62115
2006-03-31 00:00:00
34048
2006-03-31 00:00:00
64997
2006-03-31 00:00:00
34048
2006-03-31 00:00:00
64997
2006-03-31 00:00:00
34048
2006-03-31 00:00:00
64997
2006-03-31 00:00:00
41285
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
49530
2007-03-31 00:00:00
47423
2007-03-31 00:00:00
50274
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
46578
2007-03-31 00:00:00
48696
2007-03-31 00:00:00
48696
2

82767
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
81591
2011-03-31 00:00:00
78039
2011-03-31 00:00:00
79720
2011-03-31 00:00:00
76093
2011-03-31 00:00:00
76093
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
78499
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
79132
2011-03-31 00:00:00
80906
2011-03-31 00:00:00
81158
2011-03-31 00:00:00
83293
2011-03-31 00:00:00
123415
2011-03-31 00:00:00
86781
2011-03-31 00:00:00
86432
2011-03-31 00:00:00
82527
2011-03-31 00:00:00
84570
2011-03-31 00:00:00
81847
2011-03-31 00:00:00
83803
2011-03-31 00:00:00
158326
2011-03-31 00:00:00
175379
2011-03-31 00:00:00
82461
2011-03-31 00:00:00
81782
2011-03-31 00:00:00
79293
2011-03-31 00:00:00
81512
201

116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
116797
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
117176
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112290
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
112552
2015-03-31 00:00:00
166024
2015-03-31 00:00:00
117881
2015-03-31 00:00:00
112556
2015-03-31 00:00:00
118900
2015-03-31 00:00:00
118900
2015-03-31 00:00:00
118997
2015-03-31 00:00:00
118997
2015-03-31 00:00:00
118997
2015-03-31 00:00:00
115617
2015-03-31 00:00:00
112175
2015-03-31 00:00:00
117444
2015-03-31 00:00:00
106452
2015-03-31 00:00:00
1

168250
2018-03-31 00:00:00
168250
2018-03-31 00:00:00
168250
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179817
2018-03-31 00:00:00
179815
2018-03-31 00:00:00
177651
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
180031
2018-03-31 00:00:00
182825
2018-03-31 00:00:00
178061
2018-03-31 00:00:00
178061
2018-03-31 00:00:00
178061
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
177615
2018-03-31 00:00:00
180497
2018-03-31 00:00:00
180497
2018-03-31 00:00:00
168608
2018-03-31 00:00:00
168608
2018-03-31 00:00:00
1

,year_film_x,year_ceremony,ceremony,category,name,film,winner,movieId,title,genres,year_film_y,postRatings_mode,postRatings_mean
2,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False,25768,Sadie Thompson (1928),Drama,1928.0,4.0,3.500000
7,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,True,87290,Two Arabian Knights (1927),Adventure|Comedy|Romance,1927.0,3.0,2.500000
8,1927,1928,1,DIRECTING (Comedy Picture),Ted Wilde,Speedy,False,32551,Speedy (1928),Action|Comedy,1928.0,3.5,3.521739
9,1927,1928,1,ENGINEERING EFFECTS,Roy Pomeroy,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0,4.0,3.701635
10,1927,1928,1,OUTSTANDING PICTURE,Paramount Famous Lasky,Wings,True,1925,Wings (1927),Action|Drama|Romance|War,1927.0,4.0,3.701635
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,2018,2019,91,SHORT FILM (Live Action),Rodrigo Sorogoyen and María del Puy Alvarado,Mother,False,188793,Mother (2018),Thriller,2018.0,NaN,NaN
8181,2018,2019,91,SOUND EDITING,Ethan Van der Ryn and Erik Aadahl,A Quiet Place,False,185029,A Quiet Place (2018),Drama|Horror|Thriller,2018.0,NaN,NaN
8182,2018,2019,91,VISUAL EFFECTS,"Christopher Lawrence, Michael Eames, Theo Jone...",Christopher Robin,False,189783,Christopher Robin (2018),Adventure|Animation|Comedy,2018.0,NaN,NaN
8184,2018,2019,91,VISUAL EFFECTS,"Rob Bredow, Patrick Tubach, Neal Scanlan and D...",Solo: A Star Wars Story,False,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,NaN,NaN


In [5]:
import pandas as pd
import statsmodels.api as sm
dfOscarMovies = pd.read_csv('oscar_movies_postRat.csv')
dfOscarMovies = dfOscarMovies.dropna()
numOfAwards = dfOscarMovies[['movieId', 'winner']].groupby('movieId').count()
# mode of preRatings
postRat = dfOscarMovies[['movieId', 'postRatings_mode']].groupby('movieId').mean()
model = sm.OLS(postRat, numOfAwards).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:       postRatings_mode   R-squared (uncentered):                   0.550
Model:                            OLS   Adj. R-squared (uncentered):              0.549
Method:                 Least Squares   F-statistic:                              2706.
Date:                Fri, 18 Sep 2020   Prob (F-statistic):                        0.00
Time:                        10:25:27   Log-Likelihood:                         -5149.4
No. Observations:                2219   AIC:                                  1.030e+04
Df Residuals:                    2218   BIC:                                  1.031e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------